# DOWNLOADING SCRIPT

To run the other scripts or the user interface, it is essential to download the required data. The source used is: https://demanda.ree.es/visiona/peninsula/nacional/tablas/ 

Alternatively, the data can be manually downloaded from the files provided in the project. However, if data from a specific year is needed, simply execute the steps in this script and modify the corresponding year parameter.

It is important to note that during the data extraction process, a manual adjustment is required for each year: identifying the date when daylight saving time was applied in Spain. Every year, the official time is moved forward by one hour on a specific date, which necessitates data interpolation to maintain temporal consistency and ensure complete records for all days of the year.

We are going to start downloading all the data that we need, from 2020 to 2022 for training the model, and 2023 data for the testing. First, we download all the necessary libraries.

In [ ]:
import pandas as pd 
import numpy as np

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

We are going to use a loop to load the data for each year. We are going to do each year separately, because there are so many data, so I prefer to load it step by step, and have it saved independly. To load it, it is necessary to use web scraping, because all the data is posted in a url: https://demanda.ree.es/visiona/peninsula/nacional/tablas/.

The data of each day have diferent url termination, so I'm going to use anidated loops in order to iterate for each combination of year, month and day.

IMPORTANT NOTE: Currently the page has added some cookies, and this code won't work correctly. You can download manually the data from the `data` folder.

In [ ]:
columns = ['Real', 'Prevista', 'Programada', 'Date', 'Time']
df_2023 = pd.DataFrame(columns=columns)

# In order to download data for other years, you only need to change the range
for year in range(2023, 2024):
    for month in range(1, 13):
        print(month)
        for day in range(1, 32):

            if (len(str(month)) == 1):
                month = '0' + str(month)
                
            if (len(str(day)) == 1):
                day = '0' + str(day)
            url = 'https://demanda.ree.es/visiona/peninsula/nacional/tablas/'+ str(year) + '-' + str(month) + '-' + str(day) + '/1'

            # First, we set the configurations for the chrome explorer to execute in headless mode (without opening explorer window)
            chrome_options = Options()
            chrome_options.add_argument('--headless')
            chrome_options.page_load_strategy = 'normal'
            driver = webdriver.Chrome(chrome_options)

            driver.get(url)
            table = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, "//table")))

            table_html = table.get_attribute('outerHTML')
            driver.quit()
            df = pd.read_html(table_html)[0]

            if len(df) > 1:  # If the table data for that day exists
                column_names = df.iloc[0]
                df = df[1:]
                df.columns = column_names
                df.reset_index(drop=True, inplace=True)

                # In the next steps we are only taking the values until 21:00, to do not save duplicated data
                
                df[['Date', 'Time']] = df['Hora'].str.split(' ', expand=True)
                df['Date'] = pd.to_datetime(df['Date'])
                df['Time'] = df['Time'].str.replace('2A', '02')
                df = df[~df['Time'].str.contains('B')]
                df['Time'] = pd.to_datetime(df['Time']).dt.time
                df = df[0:288]
                df.drop('Hora', axis=1, inplace=True)
                df_2023 = pd.concat([df_2023, df], ignore_index=True)

1
2
3
4
5
6
7
8
9
10
11
12


Now, we need to remove the duplicated data caused by the time change. Each year, on a specific date, the official time is set back by one hour, leading to duplicate records for that period. Therefore, it is essential to identify and remove these redundant entries to ensure data consistency.

In [ ]:
# On the 29 March 2020
df_2020 = df_2020.drop_duplicates()
df_2020.to_csv('../data/df/df_2020.csv', index=False)
# On the 28 March 2021
df_2021 = df_2021.drop_duplicates()
df_2021.to_csv('../data/df/df_2021.csv', index=False)
# On the 26 March 2022
df_2022 = df_2022.drop_duplicates()
df_2022.to_csv('../data/df/df_2022.csv', index=False)
# On the 26 March 2023
df_2023 = df_2023.drop_duplicates()
df_2023.to_csv('../data/df/df_2023.csv', index=False)

In this step, we will structure the dataset into the desired format. To achieve this, we will pivot the table, ensuring that each row represents a single day and contains the corresponding values for different time intervals.

In [ ]:
df_2023 = pd.read_csv('../data/df/df_2023.csv')
df_2023 = df_2023[df_2023['Date'] != '2022-12-31'].reset_index()
df_2023 = df_2023.drop(['Programada', 'Prevista', 'index'], axis=1)

# Ensure 'Date' is in datetime format
df_2023['Date'] = pd.to_datetime(df_2023['Date'])

# Generate a unique time interval identifier for each day
# Assuming each day contains exactly 288 records (24 hours * 12 intervals per hour)
df_2023['TimeID'] = (df_2023.groupby('Date').cumcount() + 1).astype(str)

# Pivot the DataFrame to organize data by day
df_pivoted_2023 = df_2023.pivot(index='Date', columns='TimeID', values='Real')

# Rename columns to reflect time intervals
df_pivoted_2023.columns = ['5min_' + str(col) for col in df_pivoted_2023.columns]

sorted_columns = sorted(df_pivoted_2023.columns, key=lambda x: int(x.split('_')[1]))

# Sort columns numerically
df_pivoted_2023 = df_pivoted_2023[sorted_columns]

# Display the first few rows of the transformed DataFrame
df_pivoted_2023.head()

,5min_1,5min_2,5min_3,5min_4,5min_5,5min_6,5min_7,5min_8,5min_9,5min_10,...,5min_279,5min_280,5min_281,5min_282,5min_283,5min_284,5min_285,5min_286,5min_287,5min_288
Date,,,,,,,,,,,,,,,,,,,,,
2023-12-31,30.503,30.430,30.213,29.997,29.831,29.740,29.378,29.190,28.921,28.699,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01,23.548,23.578,23.612,23.495,23.393,23.363,23.293,23.330,23.215,23.208,...,25.902,25.692,25.523,25.361,24.975,24.732,24.529,24.367,24.195,23.889
2024-01-02,23.720,23.690,23.543,23.349,23.079,22.909,22.833,22.680,22.386,22.279,...,29.527,29.410,29.136,28.848,28.430,28.324,28.024,27.809,27.408,27.289
2024-01-03,26.913,26.958,26.992,26.866,26.756,26.553,26.328,26.078,25.822,25.638,...,29.737,29.372,29.091,28.680,28.532,28.289,28.130,27.903,27.618,27.372
2024-01-04,27.222,27.302,27.195,26.762,26.504,26.425,26.288,26.207,25.926,25.897,...,29.411,29.262,29.137,28.804,28.543,28.361,28.143,27.828,27.574,27.314


In the table above, we can visually observe the new structure of the dataset after the pivoting process. This transformation is essential for preparing the data for model training.

As previously mentioned, a manual adjustment is required each year to identify the date when the time change occurred (adding 1 hour). For instance, in Spain, this transition took place on March 26, 2023.

It is important to pay attention on the 2020-03-29, 2021-03-28, 2022-03-27 and 2023-03-26.

In [164]:
indx = df_pivoted_2023.index.get_loc('2023-03-26')
for i in range(287, 35, -1):
    df_pivoted_2023.iloc[indx, i] = df_pivoted_2023.iloc[indx, i-12]

df_pivoted_2023.iloc[indx, 23:35] = np.nan

Finally, we save the pivoted data locally. You will need to repeat the process with each year you want the data from.

In [ ]:
df_pivoted_2023.to_csv('../data/df_pivoted/df_pivoted_2023.csv', index=True)